In [ ]:
import pandas as pd
import numpy as np

import chardet

### File Processing

In [ ]:
# Set up the folder structure for file processing

downloadedFileFolder = './DownloadedFiles/'
stagingFolder = './Staging/'
finishedFolder = './Finished/'

##### Import characteristics files for data processing

In [ ]:
# Read in the AAU university list and save it into a CSV file

read_file = pd.read_csv (downloadedFileFolder + 'ListofUniversities.txt', header=None)
aau_univ = read_file.set_axis(['INSTNM'], axis=1)

In [ ]:
# Read the Institutional Characteristics file to get the AAU private universities' code

with open(downloadedFileFolder + 'hd2021.csv', 'rb') as f:
    enc = chardet.detect(f.read())  # or readline if the file is large

Univ_code = pd.read_csv(downloadedFileFolder + 'hd2021.csv', encoding = enc['encoding'])

# We only need institution names and codes. Drop all other columns
Univ_code.drop(Univ_code.iloc[:,2:],inplace=True, axis=1)

# Remove non AAU schools from the university code list
aau_code = Univ_code.loc[Univ_code['INSTNM'].isin(aau_univ['INSTNM'])]

In [ ]:
# A function to extract only doctoral degree data for AAU schools

def getaauphd(file, year):
    if(file.find('.csv')):
        studentData = pd.read_csv(file)
        
        #remove non aau schools from degree award data based on UNITID
        aau_studentData = studentData.loc[studentData['UNITID'].isin(aau_code['UNITID'])]
        
        # remove non doctoral degrees
        aau_doc = aau_studentData.loc[aau_studentData['AWLEVEL'] == 17]
        
    else:
        aau_doc = pd.DataFrame()
    return aau_doc

In [ ]:
# A function to fill in NaN values

def fillNa(df, fillinpair):
    #df.replace(0, np.nan, inplace=)
    for a in fillinpair:
        df[a[0]].fillna(df[a[1]], inplace=True)
        df[a[0]].astype(int)
    
    for a in fillinpair:
        df.loc[df[a[0]]== 0 , [a[0]]] = df[a[1]]
                                               
    df.fillna(0,inplace=True)
    df.replace(np.nan, 0 ,inplace=True)
    return df

##### Import student degree award data

In [ ]:
# Get the AAU doctoral degree data

aau_doc = pd.DataFrame()

cols_to_keep = []

for y in ['2011','2010','2009','2019','2020','2021']:
    
    aau_doc_y = pd.DataFrame()
    filepath =  downloadedFileFolder + 'c' + y + '_a_rv.csv'
    aau_doc_y = getaauphd(filepath, y)
    
    if y == '2011':
        
        # special process for 2011 data with a column name change.
        aau_doc_y = aau_doc_y.rename(columns={aau_doc_y.columns[63]:'CNRALW'})
        cols_to_keep = aau_doc_y.columns
        
    elif (y == '2009') or (y == '2010'):
        
        # special process for 2009 and 2010 data to remove extra columns and match with 2011 data
        
        fillinpair =[['CAIANT','DVCAIT'],['CASIAT','DVCAPT'],['CBKAAT','DVCBKT'],['CHISPT','DVCHST'],['CWHITT','DVCWHT']]
        fillNa(aau_doc_y,fillinpair)
        aau_doc_y.drop(aau_doc_y.columns.difference(cols_to_keep), axis=1, inplace=True)
    
    # Add year to check duplicates
    aau_doc_y['Year'] = y
    
    aau_doc = aau_doc.append(aau_doc_y)

In [ ]:
# Keep only the total columns and the other three searchable columns
varlist = pd.read_excel(downloadedFileFolder + 'c2021_a.xlsx', sheet_name='varlist')

varlist_total = varlist[varlist['varTitle'].str.endswith('total') | varlist['varname'].isin(['UNITID','CIPCODE','AWLEVEL'])].copy()

varlist_keep = varlist_total['varname'].copy()

new_row = pd.Series(data={'varname':'Year'}, name='x')
varlist_keep = varlist_keep.append(new_row, ignore_index = True)

# Drop not needed columns from the two files created above

aau_doc.drop(aau_doc.columns.difference(varlist_keep), axis=1, inplace=True)

## Eliminate Duplicates

In [ ]:
# Make a backup copy before eliminating duplicates
aau_doc_bk= aau_doc.copy()

# Remove duplicates
aau_doc.drop_duplicates(keep='first', inplace=True)

## Extract data based on discipline fields

In [ ]:
# A function to filter on degree codes
def getDegDataOnField(fieldfilter, degreedf, disciplineField):

    aau_doc_field = pd.DataFrame()
    for f in fieldfilter:
        aau_doc_field = aau_doc_field.append(degreedf.loc[degreedf['CIPCODE'].astype(str).str.startswith(str(f)+'.', na=False)] 
) 
    aau_doc_field['DisciplineField'] = disciplineField

    return aau_doc_field

In [ ]:
# create filter for stem degrees. Assume cipcode for stem degree starts with the following codes: 
# 11(COMPUTER AND INFORMATION SCIENCES AND SUPPORT SERVICES),14(engineering)
# 15(ENGINEERING TECHNOLOGIES AND ENGINEERING-RELATED FIELDS ), 26(biological sciences), 27(mathematics)
# 40(Physical Sciences)

stemfilter=[11,14,15,26,27,40]

aau_doc_withField = getDegDataOnField(stemfilter, aau_doc, 'STEM')

In [ ]:
# create filter for Social Sciences degrees. Assume cipcode for Social Sciences degree starts with the following codes: 
# 19(FAMILY AND CONSUMER SCIENCES/HUMAN SCIENCES),42(PSYCHOLOGY),
# 43(HOMELAND SECURITY, LAW ENFORCEMENT, FIREFIGHTING AND RELATED PROTECTIVE SERVICES)
# 44(PUBLIC ADMINISTRATION AND SOCIAL SERVICE PROFESSIONS), 45(SOCIAL SCIENCES)

ssfilter=[19,42,43,44,45]

aau_doc_withField = aau_doc_withField.append(getDegDataOnField(ssfilter, aau_doc,'Social Sciences'))

In [ ]:
# create filter for Humanities degrees. Assume cipcode for Humanities degree starts with the following codes: 
# 05(AREA, ETHNIC, CULTURAL, GENDER, AND GROUP STUDIES),16(FOREIGN LANGUAGES, LITERATURES, AND LINGUISTICS),
# 23(ENGLISH LANGUAGE AND LITERATURE/LETTERS), 24(LIBERAL ARTS AND SCIENCES, GENERAL STUDIES AND HUMANITIES)
# 38(PHILOSOPHY AND RELIGIOUS STUDIES), 50(VISUAL AND PERFORMING ARTS), 54(HISTORY)

humfilter=[5,16,23,24,38,50,54]

aau_doc_withField = aau_doc_withField.append(getDegDataOnField(humfilter, aau_doc,'Humanities'))

## Data Integrity Check

In [ ]:
# A function to check if original data total equals the sum of the columns

def checkTotal(df, columns):
    
    df['RowTotal'] = df[columns].sum(axis=1)
    df['equal']=np.where(df['CTOTALT']==df['RowTotal'], True, False)
    
    returndf = df.loc[df['equal']==False].reset_index()
    return returndf

In [ ]:
# Check files for data integrity

totalColumns = aau_doc.loc[:,(aau_doc.columns.str.endswith('T')) & (~aau_doc.columns.str.contains('TOTAL'))].columns

aau_doc_withField_check = checkTotal(aau_doc_withField, totalColumns)
print('aau_doc_withField_check is good: ' + str(aau_doc_withField_check.empty))  

## Fix data discrepancy

In [ ]:
# A function to find which item in each identified file caused the discrepancy  

def findMismatch(dftofind, dfwitherrorid, columns):
    mismatchdf = pd.DataFrame()
    for ind in dfwitherrorid.index:
        mismatchdf= mismatchdf.append(dftofind.loc[dftofind['UNITID']== dfwitherrorid['UNITID'][ind]])

    mismatchdf.drop(['CIPCODE', 'AWLEVEL'],axis=1, inplace=True)

    mismatchdf['RowTotal'] = mismatchdf[columns].sum(axis=1)
    mismatchdf['equal']=np.where(mismatchdf['CTOTALT']==mismatchdf['RowTotal'], True, False)
    
    return mismatchdf.loc[mismatchdf['equal']==False]

In [ ]:
# Find the original mismatched rows in aau_doc_withField
print(findMismatch(aau_doc_withField,aau_doc_withField_check,totalColumns))

In [ ]:
# From the above data, we assume it is a typo in CTOTALT. We reset it to 10
aau_doc_withField.loc[[162022],['CTOTALT']] = 10

In [ ]:
# From the above data, we assume it is a double-counting for CASIAT and CNHPIT. We split them to CASIAT 4 and CNHPIT 5
aau_doc_withField.loc[[137782],['CASIAT']] = 4
aau_doc_withField.loc[[137782],['CNHPIT']] = 5

## Feature Engineering

In [ ]:
# Convert year to period
aau_doc_withField.loc[aau_doc_withField['Year'].isin(['2009','2010','2011']), 'Period' ] = '2009-2011'
aau_doc_withField.loc[aau_doc_withField['Year'].isin(['2019','2020','2021']), 'Period' ] = '2019-2021'

# Drop un-needed columns (Features)
aau_doc_withField.drop(['CIPCODE','AWLEVEL','Year','RowTotal','equal'], axis=1, inplace=True)

## Sum up based on the institution

In [ ]:
# Sum up degree by race/ethnicity by the institution in three fields
sum_cols = aau_doc.loc[:,aau_doc.columns.str.endswith('T')].columns

aau_byUniv = aau_doc_withField.groupby(['DisciplineField','Period','UNITID'])[sum_cols].sum() 
aau_total = aau_byUniv.reset_index()

In [ ]:
# Add universitys' full name to the data

aau_total = pd.merge(aau_total,aau_code,on='UNITID',how='left')

## Add ex-ND average row for comparison

In [ ]:
# Add the mean of every other aau universities except ND 
aau_totalmean = aau_total.loc[aau_total['INSTNM']!='University of Notre Dame'].groupby(['DisciplineField','Period'])[sum_cols].mean().reset_index()

aau_totalmean[['UNITID',['INSTNM']]] = ['000000','AAU mean excl ND']
aau_total = aau_total.append(aau_totalmean)

In [ ]:
# Write to the excel file

aau_total.to_excel(finishedFolder + 'aaudoctoraldegrees.xlsx', index = False)